# Intro to DFT - Hands-on session


## To make this work:

1) Set up jupyter notebook tunelling to Bridges2
1) Alternatively `pip install dftpy` on your computer
2) Install VESTA on your computer 
<i>https://jp-minerals.org/vesta/en/download.html</i>


## To do:

1) Add a "markdown" cell below each code cell
2) write a description of what the cell does
3) Play around with it as we go through it!

In [ ]:
import os
import numpy as np
from dftpy.mpi import pmi, sprint, mp

In [ ]:
# from mpi4py import MPI
# mp.comm = MPI.COMM_WORLD

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
from ase.md.langevin import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md import MDLogger
from ase.io.trajectory import Trajectory
from ase import units
from dftpy.config.config import DefaultOption, OptionFormat, PrintConf
from dftpy.api.api4ase import DFTpyCalculator
from dftpy.optimization import Optimization
from dftpy.functional import Functional, TotalFunctional
from dftpy.grid import DirectGrid
from dftpy.field import DirectField
from dftpy.ions import Ions

In [ ]:
additional_files = {
    'al.lda.recpot' : 'https://raw.githubusercontent.com/EACcodes/local-pseudopotentials/master/BLPS/LDA/reci/al.lda.recpot'
}
from dftpy.formats import download_files
download_files(additional_files)

In [ ]:
size = 3 # nxnxn supercell
atoms = FaceCenteredCubic(latticeconstant=4.05, symbol="Al", size=(size, size, size), pbc=True)
len(atoms)

In [ ]:
ions = Ions.from_ase(atoms)
PP_list = {'Al': 'al.lda.recpot'}
grid = DirectGrid(ecut=20, lattice=ions.cell, mp=mp, full=False)
rho = DirectField(grid=grid)

In [ ]:
PSEUDO = Functional(type='PSEUDO', grid=grid, ions=ions, PP_list=PP_list)

In [ ]:
KE = Functional(type='KEDF', name='WT')

In [ ]:
XC = Functional(type='XC', name='LDA')

In [ ]:
HARTREE = Functional(type='HARTREE')

In [ ]:
rho[:] = ions.get_ncharges() / ions.cell.volume

In [ ]:
funcDict = {'KE' :KE, 'XC' :XC, 'HARTREE' :HARTREE, 'PSEUDO' :PSEUDO}
EnergyEvaluator = TotalFunctional(**funcDict)

In [ ]:
optimizer = Optimization(EnergyEvaluator=EnergyEvaluator, optimization_method='TN')

In [ ]:
rho = optimizer(guess_rho=rho)

# Let's play with the `DirectField` object of `DFTpy`

In [ ]:
rho.write("rho.xsf",ions=ions)

In [ ]:
rho.grid.r.shape

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rho.grid.r[2,0,0,:],rho[0,0,:])

In [ ]:
laplacian = rho.laplacian()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rho.grid.r[2,0,0,:],laplacian[0,0,:])

In [ ]:
laplacian.write("lap.xsf",ions=ions)

In [ ]:
pot_xc = XC(rho).potential

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rho.grid.r[2,0,0,:],pot_xc[0,0,:])

In [ ]:
pot_xc.write("xc.xsf",ions=ions)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rho.grid.r[2,0,0,:],rho[0,0,:],label=r"$n(z)$")
plt.plot(rho.grid.r[2,0,0,:],laplacian[0,0,:],label=r"$\nabla^2 n(z)$")
#plt.plot(rho.grid.r[2,0,0,:],pot_xc[0,0,:]+0.3,label=r"$v_{xc}(z)+0.3$")
#plt.plot(rho.grid.r[2,0,0,:],HARTREE(rho).potential[0,0,:],label=r"$v_H(z)$")
plt.legend()

### To do something a bit more involved.... compute the Hartree potential using Fourier transforms

$$
v_H(r) = \int \frac{n(r')}{|r-r'|}dr'
$$

But the above is hard to do as it would require $N^2$ operations. 

Instead, we solve it in reciprocal space recognizing that the Fourier transform ($\mathcal{F}$) of a convolution is the product of Fourier transforms

$$
\mathcal{F}[v_H(r)]= \tilde{v}_H(G) = \mathcal{F}[n]\,\,\mathcal{F}\left[\frac{1}{|r-r'|}\right] = \tilde{n}(G) \frac{4\pi}{|G|^2}
$$

then

$$
v_H(r)= \mathcal{F}^{-1}[\tilde{v}_H(G)]
$$

In [ ]:
tilde_rho = rho.fft()

In [ ]:
g2 = grid.get_reciprocal().gg

In [ ]:
g2[0,0,0]=1.0 # needed to avoid G=0 singularity - the source of many complications in PBCs

In [ ]:
tilde_v_H = tilde_rho * 4 * np.pi / g2

In [ ]:
tilde_v_H[0,0,0]=0.0 # needed to avoid G=0 singularity - the source of many complications in PBCs

In [ ]:
v_H = tilde_v_H.ifft(force_real=True)

In [ ]:
plt.plot(rho.grid.r[2,0,0,:],HARTREE(rho).potential[0,0,:],label=r"$v_H(z)$")
plt.scatter(rho.grid.r[2,0,0,:],v_H[0,0,:],label=r"Our own $v_H(z)$")
plt.legend()

# Let's play with the `DirectGrid` object of `DFTpy`

In [ ]:
grid = rho.grid

In [ ]:
grid

In [ ]:
grid.cell

In [ ]:
grid.cell[:]

In [ ]:
grid.nr

In [ ]:
grid.ecut

In [ ]:
rgrid = grid.get_reciprocal()

In [ ]:
rgrid.cell[:]

# Now, let's run a dynamics!

In [ ]:
calc = DFTpyCalculator(optimizer = optimizer, evaluator = EnergyEvaluator, rho = rho)

In [ ]:
# You could "avoid" code and simply specify an input file:
# conf = DefaultOption()
# conf["PP"]["Al"] = "al.lda.recpot"
# conf["OPT"]["method"] = "TN"
# conf["KEDF"]["kedf"] = "WT"
# conf["JOB"]["calctype"] = "Energy Force"
# conf["OUTPUT"]["time"] = False
# conf = OptionFormat(conf)
# calc = DFTpyCalculator(config=conf, mp = mp)

In [ ]:
atoms.set_calculator(calc)

In [ ]:
T = 1023 # Kelvin
np.random.seed(8888)
MaxwellBoltzmannDistribution(atoms, temperature_K = T, force_temp=True)

In [ ]:
dyn = Langevin(atoms, 2 * units.fs, temperature_K = T, friction = 0.1, trajectory='md.traj')

In [ ]:
dyn.attach(MDLogger(dyn, atoms, '-', peratom=True), interval=1)

In [ ]:
from dftpy.constants import environ
environ['LOGLEVEL'] = 3  # turn off the output

In [ ]:
dyn.run(10)

# Let's compute the RDF

In [ ]:
from ase.geometry.analysis import Analysis
analysis = Analysis(list(Trajectory('md.traj')))
rdf = analysis.get_rdf(rmax=atoms.cell.lengths().min()*0.5, nbins=100, return_dists=True)
rdf = np.asarray(rdf).mean(axis=0)

In [ ]:
np.save("Al_rdf",rdf)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rdf[1], rdf[0])
plt.xlim(1,6)

# Want to get something comparable to experiments? 

1) export this notebook as python code (.py)
2) uncomment the second cell to be able to run in parallel (notebook can't yet):
```
# from mpi4py import MPI 
# mp.comm = MPI.COMM_WORLD
```
3) increase the number of dynamics steps to at least 3000
4) Submit to the cludster!